# Token Counter Notebook

This notebook allows you to input text, retrieve outputs, and calculate token counts using OpenAI, Gemini, Anthropic, and Grok APIs.

## 1. Install and Import Required Libraries

Install necessary libraries such as `openai`, `anthropic`, and others. Import required modules like `os` and `json`.

In [1]:
# Install required libraries (uncomment if running for the first time)
# !pip install openai anthropic google-generativeai requests tiktoken

import os
import json
import openai
import anthropic
import google.generativeai as genai
import requests
import tiktoken

c:\Users\Zain_\anaconda3\envs\all_llms\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Set Up API Keys

Set up environment variables or directly define API keys for OpenAI, Gemini, Anthropic, and Grok.

In [2]:
# Set your API keys here or load them from environment variables

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "your-openai-api-key")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY", "your-anthropic-api-key")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "your-gemini-api-key")
GROK_API_KEY = os.getenv("GROK_API_KEY", "your-grok-api-key")

# Set API keys for respective libraries
openai.api_key = OPENAI_API_KEY
genai.configure(api_key=GEMINI_API_KEY)

## 3. Define Helper Functions for Token Counting

Write helper functions to calculate token counts for each API, ensuring compatibility with their respective tokenization methods.

In [3]:
def count_openai_tokens(text, model="gpt-3.5-turbo"):
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(text))

def count_gemini_tokens(text):
    # Gemini uses SentencePiece tokenizer, but for simplicity, use whitespace split as a proxy
    return len(text.split())

def count_anthropic_tokens(text):
    # Anthropic uses Claude tokenizer, which is similar to GPT-3's
    enc = tiktoken.get_encoding("cl100k_base")
    return len(enc.encode(text))

def count_grok_tokens(text):
    # Grok's tokenizer is not public; use whitespace split as a proxy
    return len(text.split())

## 4. Process Text with OpenAI API

Send the input text to the OpenAI API, retrieve the output, and calculate the token count using the helper function.

In [4]:
def process_with_openai(prompt, model="gpt-3.5-turbo"):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{"role": "user", "content": prompt}]
        )
        output = response['choices'][0]['message']['content']
        input_tokens = count_openai_tokens(prompt, model)
        output_tokens = count_openai_tokens(output, model)
        return output, input_tokens, output_tokens
    except Exception as e:
        return f"OpenAI error: {str(e)}", None, None

# Example usage:
# prompt = "Explain the theory of relativity."
# output, input_tokens, output_tokens = process_with_openai(prompt)
# print(f"Output: {output}\nInput tokens: {input_tokens}\nOutput tokens: {output_tokens}")

## 5. Process Text with Gemini API

Send the input text to the Gemini API, retrieve the output, and calculate the token count using the helper function.

In [5]:
def process_with_gemini(prompt, model="gemini-pro"):
    try:
        model_obj = genai.GenerativeModel(model)
        response = model_obj.generate_content(prompt)
        output = getattr(response, 'text', None)
        if not output:
            output = str(response)
        input_tokens = count_gemini_tokens(prompt)
        output_tokens = count_gemini_tokens(output)
        return output, input_tokens, output_tokens
    except Exception as e:
        return f"Gemini error: {str(e)}", None, None

# Example usage:
# output, input_tokens, output_tokens = process_with_gemini(prompt)
# print(f"Output: {output}\nInput tokens: {input_tokens}\nOutput tokens: {output_tokens}")

## 6. Process Text with Anthropic API

Send the input text to the Anthropic API, retrieve the output, and calculate the token count using the helper function.

In [6]:
def process_with_anthropic(prompt, model="claude-3-opus-20240229"):
    try:
        client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
        response = client.messages.create(
            model=model,
            max_tokens=512,
            messages=[{"role": "user", "content": prompt}]
        )
        output = response.content[0].text
        input_tokens = count_anthropic_tokens(prompt)
        output_tokens = count_anthropic_tokens(output)
        return output, input_tokens, output_tokens
    except Exception as e:
        return f"Anthropic error: {str(e)}", None, None

# Example usage:
# output, input_tokens, output_tokens = process_with_anthropic(prompt)
# print(f"Output: {output}\nInput tokens: {input_tokens}\nOutput tokens: {output_tokens}")

## 7. Process Text with Grok API

Send the input text to the Grok API, retrieve the output, and calculate the token count using the helper function.

In [7]:
def process_with_grok(prompt):
    try:
        url = "https://api.grok.x.ai/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {GROK_API_KEY}",
            "Content-Type": "application/json"
        }
        data = {
            "model": "grok-1",
            "messages": [{"role": "user", "content": prompt}]
        }
        response = requests.post(url, headers=headers, data=json.dumps(data), timeout=15)
        if response.status_code == 200:
            output = response.json()['choices'][0]['message']['content']
        else:
            output = f"Grok API error: {response.status_code} - {response.text}"
        input_tokens = count_grok_tokens(prompt)
        output_tokens = count_grok_tokens(output)
        return output, input_tokens, output_tokens
    except Exception as e:
        return f"Grok error: {str(e)}", None, None

# Example usage:
# output, input_tokens, output_tokens = process_with_grok(prompt)
# print(f"Output: {output}\nInput tokens: {input_tokens}\nOutput tokens: {output_tokens}")

## 8. Enter Your Prompt

Define your input text in the variable below. This prompt will be sent to all four APIs.

In [8]:
# Enter your prompt here
prompt = 'Explain the theory of relativity in simple terms.'

## 9. Run All APIs and Compare Results

This cell will send your prompt to OpenAI, Gemini, Anthropic, and Grok, then display their outputs and token counts.

In [9]:
results = {}
# OpenAI
try:
    output, in_tok, out_tok = process_with_openai(prompt)
    results['OpenAI'] = {'output': output, 'input_tokens': in_tok, 'output_tokens': out_tok}
except Exception as e:
    results['OpenAI'] = {'output': str(e), 'input_tokens': None, 'output_tokens': None}
# Gemini
try:
    output, in_tok, out_tok = process_with_gemini(prompt)
    results['Gemini'] = {'output': output, 'input_tokens': in_tok, 'output_tokens': out_tok}
except Exception as e:
    results['Gemini'] = {'output': str(e), 'input_tokens': None, 'output_tokens': None}
# Anthropic
try:
    output, in_tok, out_tok = process_with_anthropic(prompt)
    results['Anthropic'] = {'output': output, 'input_tokens': in_tok, 'output_tokens': out_tok}
except Exception as e:
    results['Anthropic'] = {'output': str(e), 'input_tokens': None, 'output_tokens': None}
# Grok
try:
    output, in_tok, out_tok = process_with_grok(prompt)
    results['Grok'] = {'output': output, 'input_tokens': in_tok, 'output_tokens': out_tok}
except Exception as e:
    results['Grok'] = {'output': str(e), 'input_tokens': None, 'output_tokens': None}
import pandas as pd
df = pd.DataFrame.from_dict(results, orient='index')
display(df)

# Optionally, print each output separately
# for api, res in results.items():
#     print(f'--- {api} ---')
#     print(f'Output: {res["output"]}\nInput tokens: {res["input_tokens"]}\nOutput tokens: {res["output_tokens"]}\n')

,output,input_tokens,output_tokens
OpenAI,OpenAI error: \n\nYou tried to access openai.C...,NaN,NaN
Gemini,Gemini error: 404 models/gemini-pro is not fou...,NaN,NaN
Anthropic,"The theory of relativity, developed by Albert ...",11.0,303.0
Grok,Grok error: HTTPSConnectionPool(host='api.grok...,NaN,NaN


In [ ]:
# Output all results and errors to a text file for easier reading
with open('api_results_dump.txt', 'w', encoding='utf-8') as f:
    for api, res in results.items():
        f.write(f'--- {api} ---\n')
        f.write(f'Output: {res["output"]}\nInput tokens: {res["input_tokens"]}\nOutput tokens: {res["output_tokens"]}\n\n')
print('Results dumped to api_results_dump.txt')

In [ ]:
# Make output cells take full width
from IPython.core.display import HTML
HTML("<style>.jp-OutputArea-output { max-width: 100vw !important; }</style>")